In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import cv2 as cv
from pathlib import Path
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
from transformers import BertTokenizer
from models.lxrt_adaptive import VQAModel_Adaptive
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
TINY_IMG_NUM = 512
from dataset.vqa import VQADataset,VQATorchDataset, VQAEvaluator
home = str(Path.home())
MSCOCO_IMGFEAT_ROOT = home + '/data/mscoco_imgfeat/'
VQA_DATA_ROOT = home+'/data/vqa/'
load_lxmert_qa_path = home+'/snap/pretrained/model'

I0209 23:59:20.785598 140487206079360 file_utils.py:35] PyTorch version 1.4.0+cpu available.
/glob/intel-python/python3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
path = VQA_DATA_ROOT
mscoco_path = MSCOCO_IMGFEAT_ROOT
splits = 'train,nominival'
tiny = True
bs = 128
shuffle = True
drop_last = True

dset = VQADataset(path,splits)
tset = VQATorchDataset(dset,mscoco_path,tiny)
evaluator = VQAEvaluator(dset)
pin_memory = True if torch.cuda.is_available() else False
data_loader = DataLoader(
        tset, batch_size=bs,
        shuffle=shuffle, num_workers=0,
        drop_last=drop_last, pin_memory=pin_memory
    )


Load 632117 data from split(s) train,nominival.
Start to load Faster-RCNN detected objects from /home/u37216/data/mscoco_imgfeat/train2014_obj36.tsv


490it [00:02, 170.15it/s]
2it [00:00, 18.74it/s]

Loaded 500 images in file /home/u37216/data/mscoco_imgfeat/train2014_obj36.tsv in 2 seconds.
Start to load Faster-RCNN detected objects from /home/u37216/data/mscoco_imgfeat/val2014_obj36.tsv


495it [00:02, 175.93it/s]


Loaded 500 images in file /home/u37216/data/mscoco_imgfeat/val2014_obj36.tsv in 2 seconds.
Use 2810 data in torch dataset



In [3]:
batch = next(iter(data_loader))

In [4]:
ques_id, feats, boxes, ques, target = batch

## Adaptive Attention Span

In [5]:
params = {'adapt_span_enabled': True, 'attn_span': 1024, 'adapt_span_loss_coeff': 0.000005, 
          'adapt_span_ramp': 32, 'adapt_span_init': 0.002, 'adapt_span_cache': True, 'nb_heads': 12, 
          'bs': 128, 'mask_size': [20,36], 'sparse_enabled': False, 'num_attention_heads': 4, 
          'layer_sizes': {'lang':9,'cross':5,'vision':5}, 'from_scratch': False,
          'layerdrop_enabled': False, 'layerdrop_num_layers': 1}

In [6]:
model = VQAModel_Adaptive(3129, params)

I0210 00:00:16.111045 140487206079360 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/u37216/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0210 00:00:16.564099 140487206079360 lxmert_utils.py:199] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/u37216/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
I0210 00:00:16.620950 140487206079360 lxmert_utils.py:207] extracting archive file /home/u37216/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /home/u37216/tmp/tmpzlt9vun1
I0210 00:00:24.256623 140487

LXRT encoder with 9 l_layers, 5 x_layers, and 5 r_layers.
Using Adaptive Variant


In [7]:
adapt = torch.load(home+'/snap/adaptive_6910.pth', map_location=torch.device('cpu'))

In [8]:
model.load_state_dict(adapt)

<All keys matched successfully>

In [11]:
output = model(feats,boxes,ques)

In [10]:
model.lxrt_encoder.model.bert.encoder.layer[8].attention

BertSelfattLayer(
  (self): BertAttention(
    (query): Linear(in_features=768, out_features=768, bias=True)
    (key): Linear(in_features=768, out_features=768, bias=True)
    (value): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (adaptive_span): AdaptiveSpan()
  )
  (output): BertAttOutput(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [45]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

I0209 22:33:52.676019 139899979099008 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/u37216/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [46]:
tokenizer.tokenize(ques[0])

['is', 'he', 'hurt', '?']

In [27]:
feats.shape, boxes.shape

(torch.Size([128, 36, 2048]), torch.Size([128, 36, 4]))